# Checking option stats and alert

Using webull to get option details like premium and greeks, then alert by email if needs investor attention. 

**How?** \
When option IV is high, usually good time to sell option for profit or hedge share positions. \
When sold option has delta too low or too high, usually indicate hedge power is too low or strong(affect profit), needs following rollover or strike price adjustment. 


**You need** \
* A dummy webull account, suggest not using the real account for trading(security...), signup using email, 1min work.
* A dummy gmail account, you need to turn on less secruity at account page. Using dummy to avoid too many traffic to mess up your regular gmail account.  

**Resources** \
* webull api: https://github.com/tedchou12/webull/blob/775f64632475999bd7ef3d06e19d4c8970b83c47/webull/webull.py#L74
* email alert: https://realpython.com/python-send-email/

**Daily usage mannual** \
* Run webull api, gmail setup each time you wanna start, (you need to pass 2-factor) 
* Update your option lists accordingly. 
* Sometimes you need to rerun the gmail setup to restart the connection if encountering email sending problem.

### Install webull api
Sometimes, pip install in cmd not working (notebook cannot import), so we directly install from notebook

In [ ]:
import sys

In [ ]:
# directly within Jupyter using the following command in a Jupyter cell
# https://stackoverflow.com/questions/50939461/cant-import-the-installed-package-in-python3-environment-of-jupyter-notebook

# first, add python to sys path
# sys.path.append('C:\\Users\\chuti\\AppData\\Roaming\\Python\\Python37\\Scripts')

# second, install 
# !{sys.executable} -m pip install webull 

### Setup webull api

In [ ]:
from webull import webull 
# from webull import paper_webull

In [ ]:
wb = webull()

In [ ]:
WB_ACCOUNT = input("Type your webull account:")
WB_PSWD = input("Type your password and press enter:")

In [ ]:
# Get temporary code for next step
wb.get_mfa(WB_ACCOUNT) 

In [ ]:
# Use mfa in email from step above (replace the one shown here)
login_info = wb.login(WB_ACCOUNT, WB_PSWD, mfa='845367')

### Setup email for alerts

In [ ]:
import smtplib, ssl

SENDER_EMAIL = input("Type your gmail:")
RECEIVER_EMAIL = SENDER_EMAIL
port = 587  # For starttls
smtp_server = "smtp.gmail.com"
password = input("Type your password and press enter:")


In [ ]:
context = ssl.create_default_context()
server = smtplib.SMTP(smtp_server, port)
server.ehlo()  # Can be omitted
server.starttls(context=context)
server.ehlo()  # Can be omitted
server.login(SENDER_EMAIL, password)

### Params and functions for alert logic

In [ ]:
# Greeks
DELTA = 'delta'
THETA = 'theta'
IV = 'impVol'

THRESHOLD_DELTA_OTM = 0.3
THRESHOLD_DELTA_ITM = 0.7

THRESHOLD_IV_HIGH = 0.6
THRESHOLD_IV_SUPER_HIGH = 1.5

NO_ALERT = ''

def delta_alert(cur):
    # put option delta is negative, so use absolute value for checking
    cur = abs(float(cur))
    # otm is too deep
    if cur < THRESHOLD_DELTA_OTM:
        return "!deep otm detected"
    # itm is too deep
    if cur > THRESHOLD_DELTA_ITM:
        return "!deep itm detected"
    
    return NO_ALERT

def iv_alert(cur):
    cur = float(cur)
    if cur > THRESHOLD_IV_SUPER_HIGH:
        return "!super high IV detected"
    if cur > THRESHOLD_IV_HIGH:
        return "!high IV detected"
    
    return NO_ALERT

class option_greeks:
    def __init__(self, delta, theta, iv):
        self.delta = delta
        self.theta = theta
        self.iv = iv
    
    def to_string(self):
        return f'delta:{self.delta}, theta:{self.theta}, iv:{self.iv}'

In [ ]:
tests_delta = [(0.2, "deep otm"), (0.8, "deep itm"), (-0.8, "deep itm"), (0.4, "normal")]
for test, expect in tests_delta:
    print(f'result:{delta_alert(test)}, expected:{expect}')
    
tests_iv = [(0.3, 'normal'), (0.8, 'high iv'), (1.6, 'super high iv')]
for test, expect in tests_iv:
    print(f'result:{iv_alert(test)}, expected:{expect}')                                                 


### Setup options you want to track

In [ ]:
### Options to track
OPTIONS = [
#    ticker, expireDate, strike, direction, thresholds
    ('TAL', '2021-07-16', '40', 'put'),
    ('TAL', '2021-07-16', '15', 'call'),
]

In [ ]:
# example of response
{'strikePrice': '40', 'call': {'volume': '0', 'latestPriceVol': '0'}, 'put': {'open': '18.51', 'high': '18.89', 'low': '18.51', 'strikePrice': '40', 'preClose': '14.90', 'openInterest': 874, 'volume': '3', 'latestPriceVol': '1', 'delta': '-0.9439', 'vega': '0.0046', 'impVol': '1.8787', 'gamma': '0.0149', 'theta': '-0.0329', 'rho': '-0.0139', 'close': '18.89', 'change': '3.99', 'changeRatio': '0.2678', 'expireDate': '2021-07-16', 'tickerId': 1020908652, 'belongTickerId': 913254311, 'openIntChange': 0, 'activeLevel': 8, 'weekly': 0, 'direction': 'put', 'derivativeStatus': 0, 'currencyId': 247, 'regionId': 6, 'exchangeId': 189, 'symbol': 'TAL210716P00040000', 'unSymbol': 'TAL', 'askList': [{'price': '18.80', 'volume': '143'}], 'bidList': [{'price': '18.40', 'volume': '69'}], 'quoteMultiplier': 100, 'quoteLotSize': 100, 'tradeTime': '2021-07-02T19:48:43.000+0000', 'tradeStamp': 1625255996000}}

In [ ]:
def alert_messages_from_options(options):
    '''
    Job to alert options at each posting
    '''
    alert_messages = []
    
    for ticker, expireDate, strike, direction in options:
        alert_message_builder = []

        option_id = f'{ticker},{expireDate},strike:{strike},{direction}'
        print(f'processing: {option_id}')

        # parse option from response
        option_details = wb.get_options_by_strike_and_expire_date(
            stock=ticker,
            expireDate=expireDate,
            strike=strike,
            direction=direction
        )[0]

        greeks = option_greeks(
            delta=option_details[direction][DELTA],
            theta=option_details[direction][THETA],
            iv=option_details[direction][IV]
        )

        # alert logics
        is_alert = False
        causes = []

        # check delta
        delta_alert_msg = delta_alert(greeks.delta)
        if delta_alert_msg:
            is_alert = True
            causes.append(delta_alert_msg)
        
        iv_alert_msg = iv_alert(greeks.iv)
        if iv_alert_msg:
            is_alert = True
            causes.append(iv_alert_msg)

        if is_alert:
            alert_message_builder.append(option_id)
            alert_message_builder.append(greeks.to_string())
            alert_message_builder.append('\t'.join(causes))
            alert_messages.append('\t'.join(alert_message_builder))
            
    return alert_messages

In [ ]:
alert_messages = alert_messages_from_options(OPTIONS)

In [ ]:
for message in alert_messages:
    print(message)

In [ ]:
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def alert_job(server, options):
    
    message = MIMEMultipart("alternative")
    message["Subject"] = "Options needs attention!"
    message["From"] = SENDER_EMAIL
    message["To"] = RECEIVER_EMAIL

    # Create the plain-text and HTML version of your message    
    alert_messages = alert_messages_from_options(options)
    text = '\n\n'.join(alert_messages)
    
    pretty_text = '<br>'.join(alert_messages)

    html = f"""\
    <html>
      <body>
        <p>
            {pretty_text}
        </p>
      </body>
    </html>
    """

    # Turn these into plain/html MIMEText objects
    part1 = MIMEText(text, "plain")
    part2 = MIMEText(html, "html")

    # Add HTML/plain-text parts to MIMEMultipart message
    # The email client will try to render the last part first
    message.attach(part1)
    message.attach(part2)

    server.sendmail(SENDER_EMAIL, RECEIVER_EMAIL, message.as_string())

In [ ]:
alert_job(server, OPTIONS)

### Sechdule the job
https://stackoverflow.com/questions/22715086/scheduling-python-script-to-run-every-hour-accurately

In [ ]:
# install package if not exist
# !{sys.executable} -m pip install apscheduler

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler

sched = BlockingScheduler()
interval = 60

@sched.scheduled_job('interval', seconds=interval)
def timed_job():
    print('fThis job is run every {interval} seconds.')
    alert_job(server, OPTIONS)
    
sched.start()